In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd

In [2]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit\
            /537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'})
data = []
base_url = "https://commons.wikimedia.org/wiki/Category:Automobiles_by_brand_by_body_style"
response = get(base_url,headers=headers)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
data.append(soup.find_all('a'))
#print(data)
#print(len(data[0]))

In [3]:
infos = []
for info in data[0]:
    infos.append(info.text)

brands = []
for brand in infos:
    if "by body style" in brand:
        brands.append(brand.split()[:2])

for br in brands:
    if 'automobiles' in br:
        br.remove('automobiles')

manufacturer = []
for el in brands:
    if len(el) == 2:
        el = el[0]+ " "+ el[1]
        manufacturer.append(el)
    else:
        for item in el:
            manufacturer.append(item)
#print(manufacturer)
#print(len(manufacturer))

In [4]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit\
            /537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'})
style = []
for car in manufacturer:
    base_url = "https://commons.wikimedia.org/wiki/Category:" + car + "_automobiles_by_body_style"
    response = get(base_url,headers=headers)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    style.append(soup.find_all('a'))
#print(style)

In [5]:
elem = []
for i in style:
    for el in i:
        elem.append(el.text.split())
#print(elem)


In [6]:
category = []
for st in elem:
    if len(st)>0:
        if st[0] in manufacturer:
            if 'automobiles' not in st and 'vehicles' not in st and "by" not in st:
                if len(st)>2:
                    ct = st[0] + "_"+ st[1] + "_"+st[2]
                    category.append(ct)
                else:
                    ct = st[0] + "_"+ st[1]
                    category.append(ct) 
#print(category)

In [7]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit\
            /537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'})
general = []
for ctr in category:
    base_url = "https://commons.wikimedia.org/wiki/Category:" + ctr
    response = get(base_url,headers=headers)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    gnrl = soup.find_all('a')
    for g in gnrl:
        if len(g) !=0:
            g_i = g.text + " " + ctr.split("_")[1]
            general.append(g_i)
#print(general)

In [8]:
general_info = []

for d in general:
    if "jpg" not in d and "automobiles" not in d and "by" not in d:
        general_info.append(d.split())
        
#print(general_info)

In [9]:
full_list = []
for e in general_info:
    if(len(e))>0:
        if e[0] in manufacturer:
            if 'automobiles' not in e and 'by' not in e:
                full_list.append(e)
            elif ".jpg" not in e:
                full_list.append(e)
#print(full_list)

In [10]:
manf = []
model = []
body_type = []
types = []

for type in category:
#l = len(type.split('_'))
    types.append(type.split('_')[1])

types=list(dict.fromkeys(types))   # remove duplicates

for car in full_list:
    manf.append(car[0])
    body_type.append(car[-1])
    model.append(" ".join(car[1:len(car)-1]))

for n, mdl in enumerate(manf):
    if mdl == "Škoda":
        manf[n] = "Skoda"
    elif mdl == "Citroën":
        manf[n] = "Skoda"

'''
print(len(manf))
print(len(model))
print(len(body_type))
print(manf)
print(model)
print(body_type)
'''

'\nprint(len(manf))\nprint(len(model))\nprint(len(body_type))\nprint(manf)\nprint(model)\nprint(body_type)\n'

In [11]:
data_1 = pd.DataFrame({'Brand': manf, 'Model': model, "Category": body_type})

In [12]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2076 entries, 0 to 2075
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Brand     2076 non-null   object
 1   Model     2076 non-null   object
 2   Category  2076 non-null   object
dtypes: object(3)
memory usage: 48.8+ KB


In [13]:
data_1.describe()

,Brand,Model,Category
count,2076,2076,2076
unique,48,2003,21
top,Toyota,stretch limousines,sedans
freq,240,5,422


In [14]:
data_1.to_csv('brands_1.csv', index=False)